<a href="https://colab.research.google.com/github/SafaSaifudeen/Medical_ASR/blob/main/Copy_of_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install Kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

# Download the dataset from Kaggle
!kaggle datasets download -d mathurinache/the-lj-speech-dataset
!unzip /content/the-lj-speech-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: LJSpeech-1.1/wavs/LJ030-0111.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0112.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0113.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0114.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0115.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0116.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0117.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0118.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0119.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0120.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0121.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0122.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0123.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0124.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0125.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0126.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0127.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0128.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0129.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-

In [ ]:
# prompt: import the saved moodel from the drive

from tensorflow import keras

# Load the saved model from your Google Drive
model = keras.models.load_model('/content/drive/MyDrive/my_model.h5', compile=False) # Assuming you saved it as 'my_model.h5'
# Or
# model = keras.models.load_model('/content/drive/MyDrive/my_model.keras', compile=False)

In [ ]:
import tensorflow as tf
import numpy as np

# Define the same parameters used during training
frame_length = 256
frame_step = 160
fft_length = 384

characters = [x for x in "abcdefghijklmnopqrstuvwxyz' "]
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)
print(f"The vocabulary = {char_to_num.get_vocabulary()} "
      f"(size ={char_to_num.vocabulary_size()})")

def preprocess_audio(file_path):
    # Read the audio file
    file = tf.io.read_file(file_path)
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    audio = tf.cast(audio, tf.float32)

    # Compute the spectrogram
    spectrogram = tf.signal.stft(audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)

    # Normalize the spectrogram
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)

    # Add a batch dimension
    spectrogram = tf.expand_dims(spectrogram, axis=0)

    return spectrogram

# Preprocess the audio file
audio_file_path = '/content/LJSpeech-1.1/wavs/LJ031-0197.wav'
input_data = preprocess_audio(audio_file_path)

The vocabulary = ['', np.str_('a'), np.str_('b'), np.str_('c'), np.str_('d'), np.str_('e'), np.str_('f'), np.str_('g'), np.str_('h'), np.str_('i'), np.str_('j'), np.str_('k'), np.str_('l'), np.str_('m'), np.str_('n'), np.str_('o'), np.str_('p'), np.str_('q'), np.str_('r'), np.str_('s'), np.str_('t'), np.str_('u'), np.str_('v'), np.str_('w'), np.str_('x'), np.str_('y'), np.str_('z'), np.str_("'"), np.str_(' ')] (size =29)


In [ ]:
from tensorflow import keras

# Load the saved model
model = keras.models.load_model('/content/drive/MyDrive/my_model.h5', compile=False)

# Make predictions
predictions = model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

# Load the saved model
model = keras.models.load_model('/content/drive/MyDrive/my_model.h5', compile=False)

# Define the same preprocessing parameters used during training
frame_length = 256
frame_step = 160
fft_length = 384

# Define character mapping - this needs to match what you used during training
characters = [x for x in "abcdefghijklmnopqrstuvwxyz' "]
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

# Function to decode predictions
def decode_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    result = tf.strings.reduce_join(num_to_char(results[0])).numpy().decode("utf-8")
    return result

# Function to preprocess a single audio file
def preprocess_audio(audio_file_path):
    # Read the audio file
    file = tf.io.read_file(audio_file_path)
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    audio = tf.cast(audio, tf.float32)

    # Create spectrogram
    spectrogram = tf.signal.stft(
        audio,
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=fft_length
    )
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)

    # Normalize
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)

    # Add batch dimension
    spectrogram = tf.expand_dims(spectrogram, 0)

    return spectrogram

# Path to your audio file
audio_file_path = '/content/LJSpeech-1.1/wavs/LJ031-0046.wav'

# Preprocess the audio
input_data = preprocess_audio(audio_file_path)

# Make prediction
prediction = model.predict(input_data)

# Decode the prediction
transcription = decode_predictions(prediction)

print("Predicted transcription:")
print(transcription)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted transcription:
andthenstnsin the prsdnshd wrss psiono thessms
